In [7]:
deployment = "gpt4"
model = "gpt-4"

# IaC：生成 Terraform 脚本

In [2]:
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

llm = AzureChatOpenAI(deployment_name=deployment, temperature=0.3, max_tokens=1000,
                     streaming=True)

memory = ConversationBufferWindowMemory(k=10) 


def get_response(input):
    conversation_with_memory = ConversationChain(
        llm=llm, 
        memory=memory,
        verbose=False
    )
    return conversation_with_memory.predict(input=input)

In [5]:
print(get_response("""
编写一个terraform scripts 创建一个AWS EKS集群
"""))

编写一个Terraform脚本来创建AWS EKS集群需要涉及到很多资源的配置，以下是一个基本的示例：

```hcl
provider "aws" {
  region = "us-west-2"
}

data "aws_ami" "eks_worker" {
  filter {
    name   = "name"
    values = ["amazon-eks-node-v*"]
  }

  most_recent = true
  owners      = ["602401143452"] # Amazon EKS AMI account ID
}

module "vpc" {
  source  = "terraform-aws-modules/vpc/aws"
  version = "2.21.0"

  name                 = "my-vpc"
  cidr                 = "10.0.0.0/16"
  azs                  = ["us-west-2a", "us-west-2b", "us-west-2c"]
  private_subnets      = ["10.0.1.0/24", "10.0.2.0/24", "10.0.3.0/24"]
  public_subnets       = ["10.0.101.0/24", "10.0.102.0/24", "10.0.103.0/24"]
  enable_nat_gateway   = true
  single_nat_gateway   = true
  enable_dns_hostnames = true

  tags = {
    "kubernetes.io/cluster/my-eks-cluster" = "shared"
  }

  public_subnet_tags = {
    "kubernetes.io/cluster/my-eks-cluster" = "shared"
    "kubernetes.io/role/elb"               = "1"
  }

  private_subnet_tags = {
    "kubernetes.io/c

In [6]:
print(get_response("""
将上面的script改写为适合阿里云的
"""))

为了将上述脚本修改为适合阿里云的，你需要更改provider和一些资源的配置。阿里云的Terraform provider是"alicloud"，并且它有自己的资源类型。以下是一个基本的修改后的示例：

```hcl
provider "alicloud" {
  access_key = "<your-access-key>"
  secret_key = "<your-secret-key>"
  region     = "cn-hangzhou"
}

data "alicloud_zones" "default" {}

data "alicloud_images" "default" {
  name_regex  = "^ubuntu_18.*_64"
  most_recent = true
  owners      = "system"
}

resource "alicloud_vpc" "vpc" {
  name       = "my-vpc"
  cidr_block = "172.16.0.0/12"
}

resource "alicloud_vswitch" "vswitch" {
  vpc_id            = alicloud_vpc.vpc.id
  cidr_block        = "172.16.0.0/21"
  availability_zone = data.alicloud_zones.default.zones.0.id
  name              = "my-vswitch"
}

resource "alicloud_security_group" "sg" {
  name   = "my-sg"
  vpc_id = alicloud_vpc.vpc.id
}

resource "alicloud_cs_managed_kubernetes" "k8s" {
  name_prefix                  = "my-k8s"
  vswitch_ids                  = [alicloud_vswitch.vswitch.id]
  new_nat_gateway              = true
  worker_instanc

# 生成应用部署文件

In [11]:
import openai
def work_on(input):
    response = openai.ChatCompletion.create(
        engine=deployment, # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a senior software engineer."},   
            {"role": "user", "content": input}
        ],
        temperature = 0.9, 
        max_tokens = 1000,
      )
    return response.choices[0].message.content

In [12]:
print(work_on("""
编写kubernetes部署文件（yaml）用于部署Nginx服务，服务端口8080
满足以下的要求
1. 部署的namespace为”service“
2. 包含readiness及liveness probe
3. 采用一个独立的service account运行
4. 包含完美终止（graceful termination）配置
"""))

以下是根据你的需求编写的Kubernetes部署文件（yaml）：

```yaml
---
apiVersion: v1
kind: Namespace
metadata:
  name: service
---
apiVersion: v1
kind: ServiceAccount
metadata:
  name: nginx-serviceaccount
  namespace: service
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  namespace: service
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      terminationGracePeriodSeconds: 30
      serviceAccountName: nginx-serviceaccount
      containers:
      - name: nginx
        image: nginx
        ports:
        - containerPort: 8080
        readinessProbe:
          tcpSocket:
            port: 8080
          initialDelaySeconds: 5
          periodSeconds: 10
        livenessProbe:
          tcpSocket:
            port: 8080
          initialDelaySeconds: 15
          periodSeconds: 20
---
apiVersion: v1
kind: Service
metadata:
  name: nginx-service
  namespace: service
spec:
  selector:
    app: ng